In [6]:
pip install xgboost


Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install lightgbm

  Using cached lightgbm-4.6.0-py3-none-win_amd64.whl.metadata (17 kB)
Using cached lightgbm-4.6.0-py3-none-win_amd64.whl (1.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install catboost

   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   - -------------------------------------- 3.7/102.4 MB 25.4 MB/s eta 0:00:04
   ---- ----------------------------------- 11.3/102.4 MB 32.6 MB/s eta 0:00:03
   -------- ------------------------------- 22.0/102.4 MB 40.5 MB/s eta 0:00:02
   ----------- ---------------------------- 29.6/102.4 MB 39.3 MB/s eta 0:00:02
   ----------------- ---------------------- 44.6/102.4 MB 46.0 MB/s eta 0:00:02
   ----------------------- ---------------- 59.8/102.4 MB 51.1 MB/s eta 0:00:01
   ----------------------------- ---------- 75.5/102.4 MB 54.9 MB/s eta 0:00:01
   ----------------------------------- ---- 90.4/102.4 MB 56.8 MB/s eta 0:00:01
   --------------------------------------  102.2/102.4 MB 58.8 MB/s eta 0:00:01
   --------------------------------------- 102.4/102.4 MB 55.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------- 9.9/9.9 MB 65.6 M

In [ ]:
pip install xgboostlss

In [ ]:
pip install optuna

In [ ]:
pip install linearmodels

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

import xgboost as xgb
from xgboost import XGBRegressor

import lightgbm as lgb
from lightgbm import LGBMRegressor

import catboost as cb
from catboost import CatBoostRegressor

from xgboostlss.model import *
from xgboostlss.distributions.Gaussian import *
from xgboostlss.datasets.data_loader import load_simulated_gaussian_data

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.feature_selection import RFE, RFECV
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.inspection import permutation_importance, PartialDependenceDisplay

import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import scipy.stats as stats

from scipy.stats import norm, poisson, gamma

import optuna
from functools import partial
import joblib
import pickle
import json
from datetime import datetime, timedelta
import time

import linearmodels as lm
from linearmodels import PanelOLS, RandomEffects

# Part 1: Data preparation

In [3]:
df_train = pd.read_csv('train_fe.csv', index_col=0)
print(df_train.shape)
print(df_train.columns)
print(df_train.head(10))
df_train.info()

(3993, 115)
Index(['Ticker', 'Nazwa2', 'rok', 'ta', 'txt', 'pi', 'str', 'xrd', 'ni',
       'ppent',
       ...
       'intan_ma', 'ppe_ma', 'sale_ma', 'cash_holdings_ma', 'roa_past',
       'lev_past', 'intan_past', 'ppe_past', 'sale_past',
       'cash_holdings_past'],
      dtype='object', length=115)
          Ticker             Nazwa2   rok         ta      txt         pi  \
0  11B PW Equity  11 bit studios SA  2005  21.127613  1.24185   6.329725   
1  11B PW Equity  11 bit studios SA  2006  21.127613  1.24185   6.329725   
2  11B PW Equity  11 bit studios SA  2007  21.127613  1.24185   6.329725   
3  11B PW Equity  11 bit studios SA  2008  21.127613  1.24185   6.329725   
4  11B PW Equity  11 bit studios SA  2009  21.127613  1.24185   6.329725   
5  11B PW Equity  11 bit studios SA  2010   1.352400 -0.05370  -0.284900   
6  11B PW Equity  11 bit studios SA  2011   2.986200  0.29240   1.585100   
7  11B PW Equity  11 bit studios SA  2012   7.336500  0.34030   1.685000   
8  11B PW 

In [4]:
df_test = pd.read_csv('test_fe.csv', index_col=0)
print(df_test.shape)
print(df_test.head(10))

(363, 115)
            Ticker                        Nazwa2   rok            ta  \
11   11B PW Equity             11 bit studios SA  2016  4.564940e+01   
23   1AT PW Equity                Atal SA/Poland  2016  1.513553e+03   
35   4FM PW Equity                 4Fun Media SA  2016  4.166300e+01   
47   AAL LN Equity            Anglo American PLC  2016  5.014900e+04   
59   ABC PW Equity                   ABC Data SA  2016  1.252895e+03   
71   ABE PW Equity                         AB SA  2016  1.045349e+03   
83   ABF LN Equity  Associated British Foods PLC  2016  1.137600e+04   
95   ABS PW Equity  Asseco Business Solutions SA  2016  3.016820e+02   
107   AC FP Equity                      Accor SA  2016  1.382241e+06   
119  ACG PW Equity                         Ac SA  2016  1.311210e+02   

             txt           pi     str     xrd            ni         ppent  \
11      2.801500    15.730800  0.1900     0.0     12.929300      0.779500   
23     15.915000   109.042999  0.1900     

In [4]:
fr = pd.read_excel('feature_ranking.xlsx', index_col=0)

In [5]:
fr.head()

,mi_score,sign_fscore,sign_fscore_0_1,corr,EN_coef,boruta_rank
rok,0.032073,1.179353e-01,0,-0.032669,0.000000e+00,19
ta,0.582922,1.464884e-03,1,0.267340,-1.404307e-07,49
txt,0.633067,5.246456e-13,1,0.368732,1.466269e-05,1
pi,0.608157,8.614688e-12,1,0.299593,8.453656e-06,3
str,0.293955,1.578384e-46,1,0.372870,NaN,9


In [6]:
fr_transposed = fr.T

print("fr variable:", fr_transposed.columns.tolist())
print("df_train variable:", df_train.columns.tolist())

fr_vars = set(fr_transposed.columns)
df_train_vars = set(df_train.columns)

print("\nOnly in fr:", fr_vars - df_train_vars)
print("\nOnly in df_train:", df_train_vars - fr_vars)

fr variable: ['rok', 'ta', 'txt', 'pi', 'str', 'xrd', 'ni', 'ppent', 'intant', 'dlc', 'dltt', 'capex', 'revenue', 'cce', 'adv', 'diff', 'roa', 'lev', 'intan', 'rd', 'ppe', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'cfc', 'dta', 'capex2_scaled', 'y_v2x_polyarchy', 'y_e_p_polity', 'y_BR_Democracy', 'WB_GDPgrowth', 'WB_GDPpc', 'WB_Inflation', 'rr_per_country', 'rr_per_sector', 'sektor_consumer discretionary', 'sektor_consumer staples', 'sektor_energy', 'sektor_health care', 'sektor_industrials', 'sektor_materials', 'sektor_real estate', 'sektor_technology', 'sektor_utilities', 'gielda_2', 'gielda_3', 'gielda_4', 'gielda_5', 'ta_log', 'txt_cat_(-63.011, -34.811]', 'txt_cat_(-34.811, 0.488]', 'txt_cat_(0.488, 24.415]', 'txt_cat_(24.415, 25.05]', 'txt_cat_(25.05, 308.55]', 'txt_cat_(308.55, 327.531]', 'txt_cat_(327.531, inf]', 'pi_cat_(-8975.0, -1.523]', 'pi_cat_(-1.523, 157.119]', 'pi_cat_(157.119, 465.9]', 'pi_cat_(465.9, 7875.5]', 'pi_cat_(7875.5, 8108.5]', 'pi_cat_(8108.5, in

# Part 2: Feature Engineering

In [7]:
df_train = df_train.drop(['Nazwa2', 'Ticker'], axis=1)

In [8]:
df = df_train.copy()

# 1. Remove duplicate rows
df = df.drop_duplicates()

# 2. Remove constant columns (zero variance)
constant_cols = [col for col in df.columns if df[col].nunique() <= 1]
df.drop(columns=constant_cols, inplace=True)

# 3. Remove columns with too many missing values
missing_ratio = df.isnull().mean()
cols_to_drop = missing_ratio[missing_ratio > 0.80].index
df.drop(columns=cols_to_drop, inplace=True)

# 4. Simple outlier handling (winsorization)
def winsorize(series, lower=0.01, upper=0.99):
    if pd.api.types.is_numeric_dtype(series):
        q_low = series.quantile(lower)
        q_high = series.quantile(upper)
        return series.clip(q_low, q_high)
    return series

for col in df.select_dtypes(include=[np.number]).columns:
    df[col] = winsorize(df[col])

# 5. Do not fill NaN for tree-based models (keep NaN)
df = df.replace("nan", np.nan)

# 6. Convert suitable columns to datetime
for col in df.columns:
    if "date" in col.lower() or "time" in col.lower():
        try:
            df[col] = pd.to_datetime(df[col], errors='ignore')
        except:
            pass

# 7. Convert object columns to category (for LightGBM/CatBoost)
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].astype("category")

# Output cleaned training set
df_cleaned = df


In [9]:
df_cleaned_vars = set(df_cleaned.columns)
print("\nOnly in fr:", fr_vars - df_cleaned_vars)


Only in fr: {'y_BR_Democracy'}


In [10]:
fr = fr.drop(['y_BR_Democracy'], axis=0)

# Part 3: Feature Selection

In [11]:
# 1. check column exists
if 'etr' not in df_cleaned.columns:
    raise KeyError("Column 'etr' not found in df_cleaned")

# 2. check for NaNs
n_nan = df_cleaned['etr'].isna().sum()

# 3. check for zeros (why MAPE -> inf)
n_zero = (df_cleaned['etr'] == 0).sum()

# 4. show a few diagnostics
min_val = df_cleaned['etr'].min()
q1 = df_cleaned['etr'].quantile(0.01)
q50 = df_cleaned['etr'].median()
q99 = df_cleaned['etr'].quantile(0.99)

# Example print (you can remove prints if you don't want intermediate output)
print(min_val, n_nan, n_zero, q1, q50, q99)


0.0 0 400 0.0 0.2050852271478223 1.0


In [12]:
from sklearn.linear_model import LinearRegression

In [28]:
import re
# ============================================================
# Step 0: Prepare data
# ============================================================

X = df_cleaned.drop(columns=['etr'], errors='ignore')
y = df_cleaned['etr']

# clean col names
X.columns = [re.sub(r'[^a-zA-Z0-9_]', '_', str(col)) for col in X.columns]
fr.index = [re.sub(r'[^a-zA-Z0-9_]', '_', str(idx)) for idx in fr.index]

# normalize fr
fr = fr.fillna(0)
fr_norm = fr.copy()
for col in fr_norm.columns:
    col_min = fr_norm[col].min()
    col_max = fr_norm[col].max()
    if col_max - col_min == 0:
        fr_norm[col] = 0
    else:
        fr_norm[col] = (fr_norm[col] - col_min) / (col_max - col_min)

fr_norm['composite_score'] = fr_norm.mean(axis=1)
fr_sorted = fr_norm.sort_values(by='composite_score', ascending=False)

# initial feature sets
all_feature_sets = {}
all_feature_sets['initial_top20'] = fr_sorted.index[:20].tolist()
all_feature_sets['initial_top30'] = fr_sorted.index[:30].tolist()
all_feature_sets['initial_top50'] = fr_sorted.index[:50].tolist()

# ============================================================
# Step 1: Feature sets from correlation
# ============================================================

if 'corr' in fr.columns:
    fr['corr_abs'] = np.abs(fr['corr'])
    corr_features = fr.sort_values('corr_abs', ascending=False).index[:20].tolist()
    all_feature_sets['corr_top20'] = corr_features

# ============================================================
# Step 2: Tree Models Feature Importance
# ============================================================

tree_models = {
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'GBM': GradientBoostingRegressor(random_state=42),
    'HistGBM': HistGradientBoostingRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42, eval_metric='rmse'),
    'LightGBM': LGBMRegressor(random_state=42, verbose=-1),
    'CatBoost': CatBoostRegressor(random_state=42, verbose=0)
}

def time_aware_permutation_importance(model, X, y):
    baseline_pred = model.predict(X)
    baseline_error = np.mean((baseline_pred - y) ** 2)
    importances = []
    for col in X.columns:
        X_perm = X.copy()
        X_perm[col] = X_perm[col].shift(1).fillna(method="bfill")
        pred = model.predict(X_perm)
        error = np.mean((pred - y) ** 2)
        importances.append(error - baseline_error)
    return np.array(importances)

valid_top50 = [f for f in all_feature_sets['initial_top50'] if f in X.columns]

for name, model in tree_models.items():
    if len(valid_top50) == 0:
        continue
    
    X_train = X[valid_top50]
    model.fit(X_train, y)

    if name == 'CatBoost':
        fi = model.get_feature_importance()
    elif name == 'HistGBM':
        fi = time_aware_permutation_importance(model, X_train, y)
    else:
        if hasattr(model, "feature_importances_"):
            fi = model.feature_importances_
        else:
            fi = time_aware_permutation_importance(model, X_train, y)

    fi_series = pd.Series(fi, index=valid_top50).sort_values(ascending=False)
    all_feature_sets[f'{name}_top20'] = fi_series.index[:20].tolist()
    all_feature_sets[f'{name}_top30'] = fi_series.index[:30].tolist()

# ============================================================
# Step 3: RFE
# ============================================================

rfe_candidates = valid_top50

if len(rfe_candidates) >= 10:
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X[rfe_candidates])
    estimator = LinearRegression()
    selector = RFECV(estimator, step=1, cv=5, min_features_to_select=5)
    selector.fit(X_scaled, y.values.ravel())

    rfe_features = [rfe_candidates[i] for i in range(len(rfe_candidates)) if selector.support_[i]]
    all_feature_sets['rfe_selected'] = rfe_features

# ============================================================
# Step 4: Permutation importance on top20
# ============================================================

if 'initial_top20' in all_feature_sets:
    perm_features = [f for f in all_feature_sets['initial_top20'] if f in X.columns]
    if len(perm_features) > 5:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X[perm_features])
        estimator = LinearRegression()
        estimator.fit(X_scaled, y.values.ravel())

        X_df_scaled = pd.DataFrame(X_scaled, columns=perm_features)
        importances = time_aware_permutation_importance(estimator, X_df_scaled, y)
        sorted_idx = np.argsort(importances)[::-1]
        all_feature_sets['perm_top10'] = [perm_features[i] for i in sorted_idx[:10]]

# ============================================================
# Step 5: Time-series CV (sMAPE)
# ============================================================

def smape(y_true, y_pred):
    return np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-8))

def time_series_cv(X, y, model, n_splits=5):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    scores = []
    for train_idx, test_idx in tscv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        scores.append(smape(y_test, y_pred))

    return np.mean(scores), np.std(scores)

# ============================================================
# Step 6: Evaluate all feature sets
# ============================================================

benchmark_model = GradientBoostingRegressor(random_state=42)
feature_set_performance = {}

for set_name, features in all_feature_sets.items():
    valid_features = [f for f in features if f in X.columns]
    if len(valid_features) < 3:
        continue

    X_subset = X[valid_features]
    mean_s, std_s = time_series_cv(X_subset, y, benchmark_model)

    feature_set_performance[set_name] = {
        'sMAPE_mean': mean_s,
        'sMAPE_std': std_s,
        'n_features': len(valid_features),
        'features': valid_features
    }

# ============================================================
# Step 7: Print clean final output only
# ============================================================

for name, metrics in feature_set_performance.items():
    print(name, metrics['sMAPE_mean'], metrics['sMAPE_std'], metrics['n_features'])

# Best set
best_set_name = min(feature_set_performance, key=lambda k: feature_set_performance[k]['sMAPE_mean'])
best_info = feature_set_performance[best_set_name]

print("BEST_FEATURE_SET:", best_set_name)
print("BEST_sMAPE:", best_info['sMAPE_mean'])
print("FEATURES:", best_info['features'])

feature_selection_results = {
    'best_set_name': best_set_name,
    'best_features': best_info['features'],
    'best_performance': best_info
}

initial_top20 0.5005981529794471 0.0439710136432261 20
initial_top30 0.5005049600725819 0.046062942201379985 30
initial_top50 0.5002790640439982 0.047948377870861994 50
corr_top20 0.49704650899010155 0.04662191037056991 20
AdaBoost_top20 0.5003003260607681 0.05186905721796153 20
AdaBoost_top30 0.5005335251384061 0.04887290552326155 30
GBM_top20 0.500213886309448 0.04682656386248585 20
GBM_top30 0.49945146632041026 0.04808253951808071 30
HistGBM_top20 0.5017056697711145 0.04597414196752422 20
HistGBM_top30 0.5002458740950759 0.04937998415006714 30
XGBoost_top20 0.5053481707393513 0.052739623468408846 20
XGBoost_top30 0.5008455724601586 0.04692004588593411 30
LightGBM_top20 0.4990701572724385 0.04882070087622782 20
LightGBM_top30 0.5006190229618467 0.047778511267940076 30
CatBoost_top20 0.5006033146554281 0.04515441257997822 20
CatBoost_top30 0.5017075361441973 0.04867670275782283 30
rfe_selected 0.5090307765929275 0.04561398978532554 13
perm_top10 0.5040453471973592 0.046797812349456056

# Part 3: Train model

In [29]:
# Step 1: Prepare training data with best feature set
X_train = X[feature_selection_results['best_features']]
y_train = y

# Step 2: Define rolling window cross-validation
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Step 3: Define all 7 models with hyperparameter grids
param_grids = {
    'AdaBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.5, 1.0],
        'loss': ['linear', 'square', 'exponential']
    },
    'GBM': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 4, 5],
        'min_samples_split': [2, 5],
        'subsample': [0.8, 1.0]
    },
    'HistGBM': {
        'max_iter': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [None, 10, 20],
        'min_samples_leaf': [1, 5, 10],
        'l2_regularization': [0, 0.1, 1.0]
    },
    'XGBoost': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 4, 5, 6],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0],
        'reg_alpha': [0, 0.1, 1.0],
        'reg_lambda': [1, 1.5, 2.0]
    },
    'LightGBM': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7, -1],
        'num_leaves': [31, 50, 100],
        'subsample': [0.8, 0.9, 1.0],
        'reg_alpha': [0, 0.1, 1.0],
        'reg_lambda': [1, 1.5, 2.0]
    },
    'CatBoost': {
        'iterations': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1],
        'depth': [4, 6, 8],
        'l2_leaf_reg': [1, 3, 5]
    },
    'XGBoostLSS': {
        'num_boost_round': [100, 200],
        'eta': [0.01, 0.05, 0.1],
        'max_depth': [3, 4, 5],
        'subsample': [0.8, 0.9]
    }
}

# Initialize models
models = {
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'GBM': GradientBoostingRegressor(random_state=42),
    'HistGBM': HistGradientBoostingRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42, eval_metric='rmse'),
    'LightGBM': LGBMRegressor(random_state=42, verbose=-1),
    'CatBoost': CatBoostRegressor(random_state=42, verbose=0),
}

# Step 4: Hyperparameter tuning with Rolling Window CV
local_champions = {}
tuning_results = {}

# Train standard models (all except XGBoostLSS)
for name, model in models.items():
    search = RandomizedSearchCV(
        model, 
        param_grids[name],
        n_iter=15,
        cv=tscv,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        random_state=42,
        verbose=0
    )
    
    search.fit(X_train, y_train)
    local_champions[name] = search.best_estimator_
    tuning_results[name] = {
        'best_params': search.best_params_,
        'best_rmse': np.sqrt(-search.best_score_)
    }

# Special handling for XGBoostLSS
print("\n[DEBUG] Starting XGBoostLSS training...")
xgblss_trained = False
xgblss_error = None

try:
    from xgboostlss.model import XGBoostLSS
    from xgboostlss.distributions.Gaussian import Gaussian
    import xgboost as xgb
    print("[DEBUG] XGBoostLSS imports successful")
    
    best_rmse = np.inf
    best_params = {}
    best_model = None
    
    # Random search for XGBoostLSS (sample 15 combinations)
    import random
    random.seed(42)
    
    all_combinations = []
    for num_rounds in param_grids['XGBoostLSS']['num_boost_round']:
        for eta in param_grids['XGBoostLSS']['eta']:
            for depth in param_grids['XGBoostLSS']['max_depth']:
                for subsample in param_grids['XGBoostLSS']['subsample']:
                    all_combinations.append({
                        'num_boost_round': num_rounds,
                        'eta': eta,
                        'max_depth': depth,
                        'subsample': subsample
                    })
    
    sampled_combinations = random.sample(all_combinations, min(15, len(all_combinations)))
    print(f"[DEBUG] Testing {len(sampled_combinations)} parameter combinations")
    
    for idx, params_combo in enumerate(sampled_combinations, 1):
        params_dict = {
            'eta': params_combo['eta'],
            'max_depth': params_combo['max_depth'],
            'subsample': params_combo['subsample']
        }
        
        rmse_scores = []
        try:
            for fold_idx, (train_idx, val_idx) in enumerate(tscv.split(X_train), 1):
                X_fold_train = X_train.iloc[train_idx]
                X_fold_val = X_train.iloc[val_idx]
                y_fold_train = y_train.iloc[train_idx]
                y_fold_val = y_train.iloc[val_idx]
                
                # Convert to numpy arrays to ensure compatibility
                X_fold_train_np = X_fold_train.values if hasattr(X_fold_train, 'values') else X_fold_train
                X_fold_val_np = X_fold_val.values if hasattr(X_fold_val, 'values') else X_fold_val
                y_fold_train_np = y_fold_train.values if hasattr(y_fold_train, 'values') else y_fold_train
                y_fold_val_np = y_fold_val.values if hasattr(y_fold_val, 'values') else y_fold_val
                
                dtrain = xgb.DMatrix(X_fold_train_np, label=y_fold_train_np)
                dval = xgb.DMatrix(X_fold_val_np, label=y_fold_val_np)
                
                model_temp = XGBoostLSS(
                    Gaussian(stabilization="None", 
                           response_fn="exp", 
                           loss_fn="nll")
                )
                
                model_temp.train(
                    params=params_dict,
                    dtrain=dtrain,
                    num_boost_round=params_combo['num_boost_round'],
                    verbose_eval=False
                )
                
                pred_params = model_temp.predict(dval, pred_type="parameters")
                y_pred = pred_params['loc']
                
                rmse = np.sqrt(mean_squared_error(y_fold_val_np, y_pred))
                rmse_scores.append(rmse)
            
            mean_rmse = np.mean(rmse_scores)
            
            if mean_rmse < best_rmse:
                best_rmse = mean_rmse
                best_params = params_combo.copy()
                
                # Train final model with best params
                X_train_np = X_train.values if hasattr(X_train, 'values') else X_train
                y_train_np = y_train.values if hasattr(y_train, 'values') else y_train
                
                best_model = XGBoostLSS(
                    Gaussian(stabilization="None", 
                           response_fn="exp", 
                           loss_fn="nll")
                )
                dtrain_full = xgb.DMatrix(X_train_np, label=y_train_np)
                best_model.train(
                    params=params_dict,
                    dtrain=dtrain_full,
                    num_boost_round=params_combo['num_boost_round'],
                    verbose_eval=False
                )
                print(f"[DEBUG] Combo {idx}/{len(sampled_combinations)}: New best RMSE = {best_rmse:.6f}")
        
        except Exception as e:
            print(f"[DEBUG] Combo {idx} failed: {str(e)[:100]}")
            continue
    
    if best_model is not None:
        local_champions['XGBoostLSS'] = best_model
        tuning_results['XGBoostLSS'] = {
            'best_params': best_params,
            'best_rmse': best_rmse
        }
        xgblss_trained = True
        print(f"[DEBUG] XGBoostLSS training completed! Best RMSE: {best_rmse:.6f}")
    else:
        print("[DEBUG] XGBoostLSS: No valid model trained")
    
except ImportError as e:
    xgblss_error = f"Import Error: {str(e)}"
    print(f"[DEBUG] {xgblss_error}")
except Exception as e:
    xgblss_error = f"Runtime Error: {type(e).__name__}: {str(e)}"
    print(f"[DEBUG] {xgblss_error}")
    import traceback
    traceback.print_exc()

# Step 5: Display final results only
print("\n" + "=" * 70)
print("FINAL HYPERPARAMETER TUNING RESULTS")
print("=" * 70)

print(f"\nTraining Data: {X_train.shape}")
print(f"Features: {len(feature_selection_results['best_features'])}")
print(f"Time Series CV: {tscv.n_splits} splits")

print("\nLOCAL CHAMPIONS - VALIDATION RMSE:")
print("-" * 50)

sorted_models = sorted(tuning_results.items(), key=lambda x: x[1]['best_rmse'])

for rank, (name, results) in enumerate(sorted_models, 1):
    print(f"{rank:2d}. {name:15} RMSE: {results['best_rmse']:.6f}")

print("\n" + "=" * 70)
print("OVERALL BEST MODEL")
print("=" * 70)

best_model_name = sorted_models[0][0]
best_model_rmse = sorted_models[0][1]['best_rmse']

print(f"Algorithm: {best_model_name}")
print(f"Validation RMSE: {best_model_rmse:.6f}")

print(f"\nTotal models trained: {len(local_champions)}")

if not xgblss_trained and xgblss_error:
    print(f"\nNote: XGBoostLSS not trained - {xgblss_error}")


[DEBUG] Starting XGBoostLSS training...
[DEBUG] XGBoostLSS imports successful
[DEBUG] Testing 15 parameter combinations
[DEBUG] Combo 1/15: New best RMSE = 0.145664
[DEBUG] Combo 11 failed: normal expects all elements of std >= 0.0
[DEBUG] Combo 13/15: New best RMSE = 0.145120
[DEBUG] XGBoostLSS training completed! Best RMSE: 0.145120

FINAL HYPERPARAMETER TUNING RESULTS

Training Data: (3993, 20)
Features: 20
Time Series CV: 5 splits

LOCAL CHAMPIONS - VALIDATION RMSE:
--------------------------------------------------
 1. XGBoostLSS      RMSE: 0.145120
 2. LightGBM        RMSE: 0.145436
 3. CatBoost        RMSE: 0.145963
 4. XGBoost         RMSE: 0.146174
 5. GBM             RMSE: 0.147862
 6. AdaBoost        RMSE: 0.148792
 7. HistGBM         RMSE: 0.149292

OVERALL BEST MODEL
Algorithm: XGBoostLSS
Validation RMSE: 0.145120

Total models trained: 7


In [34]:
import pickle
import os

# Create models directory
os.makedirs('../Advanced Ensemble Methods/models', exist_ok=True)

# Save each local champion model
for name, model in local_champions.items():
    filename = f'../Advanced Ensemble Methods/models/{name.lower()}_local_champion.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(model, f)

# Save tuning results
with open('../Advanced Ensemble Methods/models/tuning_results.pkl', 'wb') as f:
    pickle.dump(tuning_results, f)

# Save feature selection results
with open('../Advanced Ensemble Methods/models/feature_selection_results.pkl', 'wb') as f:
    pickle.dump(feature_selection_results, f)